<a href="https://colab.research.google.com/github/Nehaharish98/REST-Service-Access-with-JsonWebToken/blob/main/notebooks/EDA%26Processing_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/tvpj528mwz-1.zip -O tvpj528mwz-1.zip
!unzip -qq tvpj528mwz-1.zip
!tar -xvzf "/content/Cloud Interdatacenter Network Performance dataset/InterdatacenterNetworkPerformanceDataset.tar.gz" -C "/content/Cloud Interdatacenter Network Performance dataset/"

--2025-11-13 14:21:23--  https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/tvpj528mwz-1.zip
Resolving prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com (prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com)... 52.218.120.242, 3.5.65.219, 3.5.70.194, ...
Connecting to prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com (prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com)|52.218.120.242|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26916099 (26M) [application/zip]
Saving to: ‘tvpj528mwz-1.zip’

tvpj528mwz-1.zip    100%[===================>]  25.67M  14.2MB/s    in 1.8s    

2025-11-13 14:21:26 (14.2 MB/s) - ‘tvpj528mwz-1.zip’ saved [26916099/26916099]

PaperDataset/Azure/a2.medium_tput_udp/Fig5/8/us-east-1-a2.medium_eu-west-1-a2.medium_1433514329/us-east-1-a2.medium_eu-west-1-a2.medium_1433514329_results.json
PaperDataset/AWS/m3.xlarge_tput_tcp/EU-SA/sa-east-1a-m3.xlarge_eu-west-1a-m3.xlarge_1429194160/sa-eas

In [2]:
import os

DATA_PATH = "/content/Cloud Interdatacenter Network Performance dataset/PaperDataset/"
json_files = []

for root, dirs, files in os.walk(DATA_PATH):
    for file in files:
        full_path = os.path.join(root, file)
        if full_path.endswith(".json"):
            json_files.append(full_path)

print(f"Found {len(json_files)} JSON files.")

Found 176 JSON files.


In [3]:
import json, os, re, math
from pathlib import Path
import pandas as pd
import numpy as np
from pprint import pprint
# sanity check one file to see top-level keys or structure
with open(json_files[0], "r") as f:
    sample = json.load(f)

if isinstance(sample, list):
    print("Top-level is a list. Displaying the first element:")
    pprint(sample[0])
elif isinstance(sample, dict):
    print("Top-level keys:", list(sample.keys()))
else:
    print("Top-level is neither a list nor a dictionary. Type:", type(sample))

Top-level is a list. Displaying the first element:
{'camp': {'region_rcv': 'us-east-1',
          'region_snd': 'eu-west-1',
          'size_rcv': 'a2.medium',
          'size_snd': 'a2.medium'},
 'duration': '300',
 'exp_name': 'nuttcp_59',
 'info_rcv': {'duration': '300',
              'exp_name': 'nuttcp_59',
              'ip_agent': '10.175.234.78',
              'ip_exp_receiver': '104.41.146.138',
              'ip_exp_sender': '104.41.211.101',
              'ip_receiver': '104.41.146.138',
              'ip_sender': '104.41.211.101',
              'port': '18100',
              'protocol': 'tcp',
              'start_time': 1432340087,
              'tool': 'NUTTCP_R'},
 'info_snd': {'duration': '300',
              'exp_name': 'nuttcp_59',
              'granularity': 0.1,
              'ip_agent': '100.79.142.27',
              'ip_exp_receiver': '104.41.146.138',
              'ip_exp_sender': '104.41.211.101',
              'ip_receiver': '104.41.146.138',
              'i

In [4]:
import json

all_extracted_data = []

for json_file_path in json_files:
    # Infer provider from file path
    provider = "AWS" if "AWS" in json_file_path else ("Azure" if "Azure" in json_file_path else "Unknown")

    with open(json_file_path, "r") as f:
        from_json = json.load(f)

    # Assuming each JSON file contains a list of records
    if isinstance(from_json, list):
        for record in from_json:
            extracted_record = {
                "region_rcv": record.get("camp", {}).get("region_rcv"),
                "region_snd": record.get("camp", {}).get("region_snd"),
                "size_rcv": record.get("camp", {}).get("size_rcv"),
                "size_snd": record.get("camp", {}).get("size_snd"),
                "duration": record.get("duration"),
                "exp_name": record.get("exp_name"),
                "info_rcv": record.get("info_rcv"),
                "info_snd": record.get("info_snd"),
                "proto": record.get("proto"),
                "delay_synt": record.get("results", {}).get("delay", {}).get("synt"),
                "jitter_synt": record.get("results", {}).get("jitter", {}).get("synt"),
                "loss_synt": record.get("results", {}).get("loss", {}).get("synt"),
                "tput_synt": record.get("results", {}).get("tput", {}).get("synt"),
                "route": record.get("results", {}).get("route"),
                 "delay_detailed": record.get("results", {}).get("delay", {}).get("detailed"),
                "loss_detailed": record.get("results", {}).get("loss", {}).get("detailed"),
                "tput_detailed": record.get("results", {}).get("tput", {}).get("detailed"),
                "provider": provider # Add provider information
            }
            all_extracted_data.append(extracted_record)
    elif isinstance(from_json, dict):
         extracted_record = {
                "region_rcv": from_json.get("camp", {}).get("region_rcv"),
                "region_snd": from_json.get("camp", {}).get("region_snd"),
                "size_rcv": from_json.get("camp", {}).get("size_rcv"),
                "size_snd": from_json.get("camp", {}).get("size_snd"),
                "duration": from_json.get("duration"),
                "exp_name": from_json.get("exp_name"),
                "info_rcv": from_json.get("info_rcv"),
                "info_snd": from_json.get("info_snd"),
                "proto": from_json.get("proto"),
                "delay_synt": from_json.get("results", {}).get("delay", {}).get("synt"),
                "jitter_synt": from_json.get("results", {}).get("jitter", {}).get("synt"),
                "loss_synt": from_json.get("results", {}).get("loss", {}).get("synt"),
                "tput_synt": from_json.get("results", {}).get("tput", {}).get("synt"),
                "route": from_json.get("results", {}).get("route"),
                "delay_detailed": from_json.get("results", {}).get("delay", {}).get("detailed"),
                "loss_detailed": from_json.get("results", {}).get("loss", {}).get("detailed"),
                "tput_detailed": from_json.get("results", {}).get("tput", {}).get("detailed"),
                "provider": provider # Add provider information
            }
         all_extracted_data.append(extracted_record)


print(f"Extracted data from {len(all_extracted_data)} records.")

Extracted data from 10336 records.


In [21]:
import pandas as pd

df = pd.DataFrame(all_extracted_data)

# Expand the 'delay_synt' column
delay_synt_df = pd.json_normalize(df['delay_synt'], sep='_')
delay_synt_df.columns = ['delay_' + col for col in delay_synt_df.columns]

# Expand the 'jitter_synt' column
jitter_synt_df = pd.json_normalize(df['jitter_synt'], sep='_')
jitter_synt_df.columns = ['jitter_' + col for col in jitter_synt_df.columns]

# Expand the 'loss_synt' column
loss_synt_df = pd.json_normalize(df['loss_synt'], sep='_')
loss_synt_df.columns = ['loss_' + col for col in loss_synt_df.columns]

# Expand the 'tput_synt' column
tput_synt_df = pd.json_normalize(df['tput_synt'], sep='_')
tput_synt_df.columns = ['tput_' + col for col in tput_synt_df.columns]

# Expand the 'info_rcv' column
info_rcv_df = pd.json_normalize(df['info_rcv'], sep='_')
info_rcv_df.columns = ['info_rcv_' + col for col in info_rcv_df.columns]

# Expand the 'info_snd' column
info_snd_df = pd.json_normalize(df['info_snd'], sep='_')
info_snd_df.columns = ['info_snd_' + col for col in info_snd_df.columns]

# Expand the 'route' column
route_df = pd.json_normalize(df['route'], sep='_')
route_df.columns = ['route_' + col for col in route_df.columns]


# Drop the original nested columns
df = df.drop(columns=['delay_synt', 'jitter_synt', 'loss_synt', 'tput_synt', 'info_rcv', 'info_snd', 'route'])

# Concatenate the new DataFrames with the original DataFrame
df = pd.concat([df, delay_synt_df, jitter_synt_df, loss_synt_df, tput_synt_df, info_rcv_df, info_snd_df, route_df], axis=1)

display(df.head())

,region_rcv,region_snd,size_rcv,size_snd,duration,exp_name,proto,delay_detailed,loss_detailed,tput_detailed,...,info_snd_ip_agent,info_snd_port,info_snd_granularity,info_snd_duration,info_snd_ip_exp_sender,info_snd_ip_sender,info_snd_tool,info_snd_ip_exp_receiver,route_detailed,route_synt
0,us-east-1,eu-west-1,a2.medium,a2.medium,300,nuttcp_59,tcp,None,"{'1432340093': 0.0, '1432340094': 0.0, '143234...","{'1432340093': 147.76221818181818, '1432340094...",...,100.79.142.27,18100,0.1,300,104.41.211.101,104.41.211.101,NUTTCP_S,104.41.146.138,NaN,NaN
1,us-east-1,eu-west-1,a2.medium,a2.medium,300,nuttcp_58,tcp,None,"{'1432339116': 0.0, '1432339117': 0.0, '143233...","{'1432339116': 150.6188, '1432339117': 280.774...",...,100.79.142.27,18100,0.1,300,104.41.211.101,104.41.211.101,NUTTCP_S,104.41.146.138,NaN,NaN
2,us-east-1,eu-west-1,a2.medium,a2.medium,300,nuttcp_53,tcp,None,"{'1432334336': 0.0, '1432334337': 0.0, '143233...","{'1432334336': 262.1406, '1432334337': 265.296...",...,100.79.142.27,18100,0.1,300,104.41.211.101,104.41.211.101,NUTTCP_S,104.41.146.138,NaN,NaN
3,us-east-1,eu-west-1,a2.medium,a2.medium,300,nuttcp_52,tcp,None,"{'1432333312': 0.0, '1432333313': 0.0, '143233...","{'1432333312': 266.81963, '1432333313': 269.51...",...,100.79.142.27,18100,0.1,300,104.41.211.101,104.41.211.101,NUTTCP_S,104.41.146.138,NaN,NaN
4,us-east-1,eu-west-1,a2.medium,a2.medium,300,nuttcp_51,tcp,None,"{'1432332288': 0.0, '1432332289': 0.0, '143233...","{'1432332288': 266.86212, '1432332289': 264.76...",...,100.79.142.27,18100,0.1,300,104.41.211.101,104.41.211.101,NUTTCP_S,104.41.146.138,NaN,NaN


In [6]:
for col in df.columns:
    if 'detailed' not in col:
        print(f"\nUnique values in {col}:")
        try:
            # Attempt to print unique values, handle potential errors with unhashable types
            print(df[col].unique())
        except TypeError:
            print(f"  Could not print unique values for {col} due to unhashable types.")


Unique values in region_rcv:
['us-east-1' 'eu-west-1' 'sa-east-1' 'ap-southeast-1' 'us-east-1d'
 'eu-west-1c' 'us-east-1a' 'eu-west-1a' 'us-east-1b' 'eu-west-1b'
 'sa-east-1c' 'sa-east-1b' 'sa-east-1a' 'ap-southeast-1b'
 'ap-southeast-1a' None]

Unique values in region_snd:
['eu-west-1' 'us-east-1' 'sa-east-1' 'ap-southeast-1' 'eu-west-1c'
 'us-east-1d' 'eu-west-1a' 'us-east-1a' 'eu-west-1b' 'us-east-1b'
 'sa-east-1c' 'sa-east-1a' 'sa-east-1b' 'ap-southeast-1a'
 'ap-southeast-1b' None]

Unique values in size_rcv:
['a2.medium' 'a4.xlarge' 'm3.xlarge' 'm3.medium' 'medium' None]

Unique values in size_snd:
['a2.medium' 'a4.xlarge' 'm3.xlarge' 'm3.medium' 'medium' None]

Unique values in duration:
['300' None '120' '480']

Unique values in exp_name:
['nuttcp_59' 'nuttcp_58' 'nuttcp_53' 'nuttcp_52' 'nuttcp_51' 'nuttcp_50'
 'nuttcp_57' 'nuttcp_56' 'nuttcp_55' 'nuttcp_54' 'nuttcp_48' 'nuttcp_49'
 'nuttcp_44' 'nuttcp_45' 'nuttcp_46' 'nuttcp_47' 'nuttcp_40' 'nuttcp_41'
 'nuttcp_42' 'nuttcp_43'

In [27]:
non_detailed_cols = [col for col in df.columns if 'detailed' not in col]

In [28]:
column_summary = []

for col in non_detailed_cols:
    num_unique = df[col].nunique()
    sample_unique = df[col].unique()[:5].tolist()  # Get the first 5 unique values as a list
    column_summary.append({
        'Column Name': col,
        'Number of Unique Values': num_unique,
        'Sample Unique Values': sample_unique
    })

In [9]:
summary_df = pd.DataFrame(column_summary)
display(summary_df)

,Column Name,Number of Unique Values,Sample Unique Values
0,region_rcv,15,"[us-east-1, eu-west-1, sa-east-1, ap-southeast..."
1,region_snd,15,"[eu-west-1, us-east-1, sa-east-1, ap-southeast..."
2,size_rcv,5,"[a2.medium, a4.xlarge, m3.xlarge, m3.medium, m..."
3,size_snd,5,"[a2.medium, a4.xlarge, m3.xlarge, m3.medium, m..."
4,duration,3,"[300, None, 120, 480]"
...,...,...,...
57,info_snd_ip_exp_sender,151,"[104.41.211.101, 104.41.146.138, 104.41.201.21..."
58,info_snd_ip_sender,151,"[104.41.211.101, 104.41.146.138, 104.41.201.21..."
59,info_snd_tool,2,"[NUTTCP_S, NUTTCP_R, nan]"
60,info_snd_ip_exp_receiver,151,"[104.41.146.138, 104.41.211.101, 104.41.45.136..."


In [10]:
import pandas as pd

df = pd.DataFrame(all_extracted_data)

In [11]:
df.shape

(10336, 18)

In [12]:
df[df.duration.isna()]

,region_rcv,region_snd,size_rcv,size_snd,duration,exp_name,info_rcv,info_snd,proto,delay_synt,jitter_synt,loss_synt,tput_synt,route,delay_detailed,loss_detailed,tput_detailed,provider
7768,None,None,None,None,None,paris_1,None,None,None,None,None,None,None,None,None,None,None,AWS
7770,None,None,None,None,None,paris_2,None,None,None,None,None,None,None,None,None,None,None,AWS
7771,None,None,None,None,None,paris_3,None,None,None,None,None,None,None,None,None,None,None,AWS
7772,None,None,None,None,None,paris_4,None,None,None,None,None,None,None,None,None,None,None,AWS
7774,None,None,None,None,None,paris_5,None,None,None,None,None,None,None,None,None,None,None,AWS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8603,None,None,None,None,None,paris_26,None,None,None,None,None,None,None,None,None,None,None,AWS
8604,None,None,None,None,None,paris_27,None,None,None,None,None,None,None,None,None,None,None,AWS
8605,None,None,None,None,None,paris_28,None,None,None,None,None,None,None,None,None,None,None,AWS
8606,None,None,None,None,None,paris_29,None,None,None,None,None,None,None,None,None,None,None,AWS


In [13]:
df.dropna(subset=['duration'], inplace=True)
print(f"Shape of DataFrame after dropping rows with NaN in 'duration': {df.shape}")

Shape of DataFrame after dropping rows with NaN in 'duration': (9616, 18)


**Checking for percentage of rows for each unique duration **

In [15]:
duration_percentages = df['duration'].value_counts(normalize=True) * 100
print("Percentage of rows for each unique duration:")
print(duration_percentages.to_string())

Percentage of rows for each unique duration:
duration
300    98.752080
120     0.748752
480     0.499168


In [16]:
df = df[~df['duration'].isin(['120', '480'])].copy()
print(f"Shape of DataFrame after dropping rows with duration '120' and '480': {df.shape}")

Shape of DataFrame after dropping rows with duration '120' and '480': (9496, 18)


In [17]:
print(df['duration'].unique())

['300']


In [22]:
for column in df.columns:
    if column.startswith('delay_'):
        print(f"Unique values in column '{column}':")
        # Check if the column contains dictionaries
        if any(isinstance(x, dict) for x in df[column].dropna()):
            print(df[column].apply(lambda x: str(x) if isinstance(x, dict) else x).unique())
        else:
            print(df[column].unique())
        print("\n" + "-"*50 + "\n")

Unique values in column 'delay_detailed':
[None]

--------------------------------------------------

Unique values in column 'delay_std':
[nan]

--------------------------------------------------

Unique values in column 'delay_pctl5':
[nan]

--------------------------------------------------

Unique values in column 'delay_min':
[nan]

--------------------------------------------------

Unique values in column 'delay_max':
[nan]

--------------------------------------------------

Unique values in column 'delay_pctl25':
[nan]

--------------------------------------------------

Unique values in column 'delay_pctl95':
[nan]

--------------------------------------------------

Unique values in column 'delay_median':
[nan]

--------------------------------------------------

Unique values in column 'delay_pctl75':
[nan]

--------------------------------------------------

Unique values in column 'delay_mean':
[77.87 76.85 81.26 ... 51.87 52.42 50.72]

-----------------------------------

In [23]:
columns_to_drop = [col for col in df.columns if col.startswith('delay_') and col != 'delay_mean']
df.drop(columns=columns_to_drop, inplace=True)
print(f"Shape of DataFrame after dropping 'delay_' columns (excluding 'delay_mean'): {df.shape}")

Shape of DataFrame after dropping 'delay_' columns (excluding 'delay_mean'): (10336, 57)


In [24]:
print(df.shape)

(10336, 57)


In [29]:
import pandas as pd

# Recreate df from all_extracted_data to ensure a fresh start
df = pd.DataFrame(all_extracted_data)

# Expand the 'delay_synt' column
delay_synt_df = pd.json_normalize(df['delay_synt'], sep='_')
delay_synt_df.columns = ['delay_' + col for col in delay_synt_df.columns]

# Expand the 'jitter_synt' column
jitter_synt_df = pd.json_normalize(df['jitter_synt'], sep='_')
jitter_synt_df.columns = ['jitter_' + col for col in jitter_synt_df.columns]

# Expand the 'loss_synt' column
loss_synt_df = pd.json_normalize(df['loss_synt'], sep='_')
loss_synt_df.columns = ['loss_' + col for col in loss_synt_df.columns]

# Expand the 'tput_synt' column
tput_synt_df = pd.json_normalize(df['tput_synt'], sep='_')
tput_synt_df.columns = ['tput_' + col for col in tput_synt_df.columns]

# Expand the 'info_rcv' column
info_rcv_df = pd.json_normalize(df['info_rcv'], sep='_')
info_rcv_df.columns = ['info_rcv_' + col for col in info_rcv_df.columns]

# Expand the 'info_snd' column
info_snd_df = pd.json_normalize(df['info_snd'], sep='_')
info_snd_df.columns = ['info_snd_' + col for col in info_snd_df.columns]

# Expand the 'route' column
route_df = pd.json_normalize(df['route'], sep='_')
route_df.columns = ['route_' + col for col in route_df.columns]

# Drop the original nested columns (only synt are expanded, detailed remain in their original column type)
df = df.drop(columns=['delay_synt', 'jitter_synt', 'loss_synt', 'tput_synt', 'info_rcv', 'info_snd', 'route'])

# Concatenate the new DataFrames with the original DataFrame
df = pd.concat([df, delay_synt_df, jitter_synt_df, loss_synt_df, tput_synt_df, info_rcv_df, info_snd_df, route_df], axis=1)

print(f"Shape after expanding nested JSON columns: {df.shape}")

Shape after expanding nested JSON columns: (10336, 66)


In [30]:
# Convert duration to numeric and drop rows with NaN values in 'duration'
df['duration'] = pd.to_numeric(df['duration'], errors='coerce')
df.dropna(subset=['duration'], inplace=True)
print(f"Shape after dropping NaN in 'duration': {df.shape}")

# Drop rows where duration is '120' or '480'
df = df[~df['duration'].isin([120, 480])].copy()
print(f"Shape after dropping duration '120' and '480': {df.shape}")

Shape after dropping NaN in 'duration': (9616, 66)
Shape after dropping duration '120' and '480': (9496, 66)


In [31]:
# Drop all 'delay_' columns except 'delay_mean'
columns_to_drop = [col for col in df.columns if col.startswith('delay_') and col != 'delay_mean']
df.drop(columns=columns_to_drop, inplace=True)
print(f"Final shape of DataFrame: {df.shape}")

Final shape of DataFrame: (9496, 57)


In [33]:
display(df.head())

,region_rcv,region_snd,size_rcv,size_snd,duration,exp_name,proto,loss_detailed,tput_detailed,provider,...,info_snd_ip_agent,info_snd_port,info_snd_granularity,info_snd_duration,info_snd_ip_exp_sender,info_snd_ip_sender,info_snd_tool,info_snd_ip_exp_receiver,route_detailed,route_synt
0,us-east-1,eu-west-1,a2.medium,a2.medium,300.0,nuttcp_59,tcp,"{'1432340093': 0.0, '1432340094': 0.0, '143234...","{'1432340093': 147.76221818181818, '1432340094...",Azure,...,100.79.142.27,18100,0.1,300,104.41.211.101,104.41.211.101,NUTTCP_S,104.41.146.138,NaN,NaN
1,us-east-1,eu-west-1,a2.medium,a2.medium,300.0,nuttcp_58,tcp,"{'1432339116': 0.0, '1432339117': 0.0, '143233...","{'1432339116': 150.6188, '1432339117': 280.774...",Azure,...,100.79.142.27,18100,0.1,300,104.41.211.101,104.41.211.101,NUTTCP_S,104.41.146.138,NaN,NaN
2,us-east-1,eu-west-1,a2.medium,a2.medium,300.0,nuttcp_53,tcp,"{'1432334336': 0.0, '1432334337': 0.0, '143233...","{'1432334336': 262.1406, '1432334337': 265.296...",Azure,...,100.79.142.27,18100,0.1,300,104.41.211.101,104.41.211.101,NUTTCP_S,104.41.146.138,NaN,NaN
3,us-east-1,eu-west-1,a2.medium,a2.medium,300.0,nuttcp_52,tcp,"{'1432333312': 0.0, '1432333313': 0.0, '143233...","{'1432333312': 266.81963, '1432333313': 269.51...",Azure,...,100.79.142.27,18100,0.1,300,104.41.211.101,104.41.211.101,NUTTCP_S,104.41.146.138,NaN,NaN
4,us-east-1,eu-west-1,a2.medium,a2.medium,300.0,nuttcp_51,tcp,"{'1432332288': 0.0, '1432332289': 0.0, '143233...","{'1432332288': 266.86212, '1432332289': 264.76...",Azure,...,100.79.142.27,18100,0.1,300,104.41.211.101,104.41.211.101,NUTTCP_S,104.41.146.138,NaN,NaN


In [40]:
for col_name in ['route_detailed', 'route_synt']:
    if col_name in df.columns:
        print(f"Unique values in column '{col_name}':")
        # Check if the column contains dictionaries before trying to make them unique
        if any(isinstance(x, dict) for x in df[col_name].dropna()):
            print(df[col_name].apply(lambda x: str(x) if isinstance(x, dict) else x).unique())
        else:
            print(df[col_name].unique())
        print("\n" + "-"*50 + "\n")
    else:
        print(f"Column '{col_name}' not found in DataFrame.\n" + "-"*50 + "\n")

Column 'route_detailed' not found in DataFrame.
--------------------------------------------------

Column 'route_synt' not found in DataFrame.
--------------------------------------------------



In [37]:
columns_to_drop_route = ['route_detailed', 'route_synt']
df.drop(columns=columns_to_drop_route, inplace=True)
print(f"Shape of DataFrame after dropping 'route_detailed' and 'route_synt': {df.shape}")

Shape of DataFrame after dropping 'route_detailed' and 'route_synt': (9496, 55)


In [42]:
print(f"Unique values in column 'region_rcv':\n{df['region_rcv'].unique()}\n")
print(f"Unique values in column 'region_snd':\n{df['region_snd'].unique()}")

Unique values in column 'region_rcv':
['us-east-1' 'eu-west-1' 'sa-east-1' 'ap-southeast-1' 'us-east-1d'
 'eu-west-1c' 'us-east-1a' 'eu-west-1a' 'us-east-1b' 'eu-west-1b'
 'sa-east-1c' 'sa-east-1b' 'sa-east-1a' 'ap-southeast-1b'
 'ap-southeast-1a']

Unique values in column 'region_snd':
['eu-west-1' 'us-east-1' 'sa-east-1' 'ap-southeast-1' 'eu-west-1c'
 'us-east-1d' 'eu-west-1a' 'us-east-1a' 'eu-west-1b' 'us-east-1b'
 'sa-east-1c' 'sa-east-1a' 'sa-east-1b' 'ap-southeast-1a'
 'ap-southeast-1b']


In [41]:
print(df.columns.tolist())

['region_rcv', 'region_snd', 'size_rcv', 'size_snd', 'duration', 'exp_name', 'proto', 'loss_detailed', 'tput_detailed', 'provider', 'delay_mean', 'loss_std', 'loss_pctl5', 'loss_min', 'loss_max', 'loss_pctl25', 'loss_pctl95', 'loss_median', 'loss_pctl75', 'loss_mean', 'tput_std', 'tput_pctl5', 'tput_min', 'tput_max', 'tput_pctl25', 'tput_pctl95', 'tput_median', 'tput_pctl75', 'tput_mean', 'info_rcv_protocol', 'info_rcv_ip_receiver', 'info_rcv_exp_name', 'info_rcv_start_time', 'info_rcv_ip_agent', 'info_rcv_port', 'info_rcv_duration', 'info_rcv_ip_exp_sender', 'info_rcv_ip_sender', 'info_rcv_tool', 'info_rcv_ip_exp_receiver', 'info_rcv_target_bwd', 'info_rcv_granularity', 'info_snd_protocol', 'info_snd_ip_receiver', 'info_snd_exp_name', 'info_snd_start_time', 'info_snd_target_bwd', 'info_snd_ip_agent', 'info_snd_port', 'info_snd_granularity', 'info_snd_duration', 'info_snd_ip_exp_sender', 'info_snd_ip_sender', 'info_snd_tool', 'info_snd_ip_exp_receiver']


In [44]:
import re
import pandas as pd
import numpy as np

AWS_REGION_RE = re.compile(r"^(?P<pfx>[a-z]{2})-(?P<area>[a-z]+)-(?P<num>\d+)(?P<az>[a-z])?$")

def split_region_aws(x: str):
    """Return (base_region, az) or (x, None) if pattern doesn't match."""
    if not isinstance(x, str) or not x.strip():
        return (np.nan, None)
    x = x.strip().lower()
    m = AWS_REGION_RE.match(x)
    if not m:
        # leave non-AWS-style labels untouched as base, no az
        return (x, None)
    base = f"{m.group('pfx')}-{m.group('area')}-{m.group('num')}"
    az = m.group('az')
    return (base, az)

def continent_from_region(base: str):
    if not isinstance(base, str): return np.nan
    if base.startswith("us-"): return "americas"
    if base.startswith("sa-"): return "americas"
    if base.startswith("eu-"): return "europe"
    if base.startswith("ap-"): return "asia-pacific"
    if base.startswith("af-"): return "africa"
    if base.startswith("me-"): return "middle-east"
    return "other"

# Apply to your columns
for col in ["region_snd", "region_rcv"]:
    base_col = col + "_base"
    az_col   = ("az_snd" if col=="region_snd" else "az_rcv")
    bases, azs = [], []
    for v in df[col].astype("string"):
        b, a = split_region_aws(v if v is not pd.NA else None)
        bases.append(b); azs.append(a)
    df[base_col] = pd.Series(bases, dtype="string")
    df[az_col]   = pd.Series(azs, dtype="string")

# Helpful booleans and pair key (use *_base)
df["same_region"] = df["region_snd_base"].eq(df["region_rcv_base"])

df["snd_continent"] = df["region_snd_base"].apply(continent_from_region).astype("string")
df["rcv_continent"] = df["region_rcv_base"].apply(continent_from_region).astype("string")
df["same_continent"] = df["snd_continent"].eq(df["rcv_continent"])

# Stable pair key for grouping/splits
if "provider" in df.columns:
    df["pair_base"] = (
        df["provider"].astype("string") + ":" +
        df["region_snd_base"].astype("string").fillna('UNKNOWN') + "->" +
        df["region_rcv_base"].astype("string").fillna('UNKNOWN')
    )
else:
    df["pair_base"] = (
        df["region_snd_base"].astype("string").fillna('UNKNOWN') + "->" +
        df["region_rcv_base"].astype("string").fillna('UNKNOWN')
    )

In [45]:
print("Unique (raw) region_snd:", df["region_snd"].nunique())
print("Unique (base) region_snd_base:", df["region_snd_base"].nunique())

print(df[["region_snd","region_snd_base","az_snd"]].drop_duplicates().head(10))
print(df[["region_rcv","region_rcv_base","az_rcv"]].drop_duplicates().head(10))


Unique (raw) region_snd: 15
Unique (base) region_snd_base: 4
          region_snd region_snd_base az_snd
0          eu-west-1       eu-west-1   <NA>
96         us-east-1       us-east-1   <NA>
216        sa-east-1       sa-east-1   <NA>
240   ap-southeast-1  ap-southeast-1   <NA>
856       eu-west-1c       eu-west-1      c
952       us-east-1d       us-east-1      d
1048      eu-west-1a       eu-west-1      a
1144      us-east-1a       us-east-1      a
1240      eu-west-1b       eu-west-1      b
1336      us-east-1b       us-east-1      b
          region_rcv region_rcv_base az_rcv
0          us-east-1       us-east-1   <NA>
96         eu-west-1       eu-west-1   <NA>
192        sa-east-1       sa-east-1   <NA>
264   ap-southeast-1  ap-southeast-1   <NA>
856       us-east-1d       us-east-1      d
952       eu-west-1c       eu-west-1      c
1048      us-east-1a       us-east-1      a
1144      eu-west-1a       eu-west-1      a
1240      us-east-1b       us-east-1      b
1336      eu-we

In [46]:
print(df.columns.tolist())


['region_rcv', 'region_snd', 'size_rcv', 'size_snd', 'duration', 'exp_name', 'proto', 'loss_detailed', 'tput_detailed', 'provider', 'delay_mean', 'loss_std', 'loss_pctl5', 'loss_min', 'loss_max', 'loss_pctl25', 'loss_pctl95', 'loss_median', 'loss_pctl75', 'loss_mean', 'tput_std', 'tput_pctl5', 'tput_min', 'tput_max', 'tput_pctl25', 'tput_pctl95', 'tput_median', 'tput_pctl75', 'tput_mean', 'info_rcv_protocol', 'info_rcv_ip_receiver', 'info_rcv_exp_name', 'info_rcv_start_time', 'info_rcv_ip_agent', 'info_rcv_port', 'info_rcv_duration', 'info_rcv_ip_exp_sender', 'info_rcv_ip_sender', 'info_rcv_tool', 'info_rcv_ip_exp_receiver', 'info_rcv_target_bwd', 'info_rcv_granularity', 'info_snd_protocol', 'info_snd_ip_receiver', 'info_snd_exp_name', 'info_snd_start_time', 'info_snd_target_bwd', 'info_snd_ip_agent', 'info_snd_port', 'info_snd_granularity', 'info_snd_duration', 'info_snd_ip_exp_sender', 'info_snd_ip_sender', 'info_snd_tool', 'info_snd_ip_exp_receiver', 'region_snd_base', 'az_snd', 're

In [47]:
# 1) use base regions for grouping everywhere
PAIR_KEYS = ["provider","region_snd_base","region_rcv_base"]  # was: provider + raw regions

# 2) when selecting features for models, prefer base + optional AZs
BASE_CATS = ["provider","region_snd_base","region_rcv_base","proto","size_snd","size_rcv"]
OPTIONAL_CATS = ["az_snd","az_rcv","same_region","same_continent"]  # include if you want

# Example: before fitting a model, drop raw region cols to avoid redundancy
def select_model_X(df, extra_numeric=None, include_az=True):
    drop_dupes = ["region_snd","region_rcv"]  # raw columns
    X = df.drop(columns=[c for c in drop_dupes if c in df.columns], errors="ignore").copy()
    # choose categoricals
    cats = BASE_CATS + (OPTIONAL_CATS if include_az else [])
    cats = [c for c in cats if c in X.columns]
    # make sure dtype is string/bool for OHE
    for c in cats:
        if c in X.columns:
            if X[c].dtype.name not in ("string","bool"):
                X[c] = X[c].astype("string")
    # numeric extras if you pass them (e.g., lags)
    if extra_numeric:
        for c in extra_numeric:
            if c in X.columns:
                X[c] = pd.to_numeric(X[c], errors="coerce")
    return X


In [48]:
# 1) confirm collapse worked
print("pairs (raw):", df.groupby(["provider","region_snd","region_rcv"]).size().shape[0])
print("pairs (base):", df.groupby(["provider","region_snd_base","region_rcv_base"]).size().shape[0])

# 2) ensure no AZs leaked into grouping keys
assert df["region_snd_base"].str.contains(r"[a-z]$", na=False).sum() == 0
assert df["region_rcv_base"].str.contains(r"[a-z]$", na=False).sum() == 0

# 3) check data per pair (should increase after collapsing)
by_pair = df.groupby(["provider","region_snd_base","region_rcv_base"]).size().rename("n").sort_values(ascending=False)
by_pair.head(10)


pairs (raw): 52
pairs (base): 24


provider  region_snd_base  region_rcv_base
AWS       ap-southeast-1   eu-west-1          864
          eu-west-1        ap-southeast-1     864
          sa-east-1        us-east-1          792
          us-east-1        sa-east-1          672
          eu-west-1        sa-east-1          576
                           us-east-1          576
          ap-southeast-1   sa-east-1          576
                           us-east-1          576
          sa-east-1        eu-west-1          576
                           ap-southeast-1     576
Name: n, dtype: int64

In [49]:
print(df['provider'].unique())

['Azure' 'AWS']


In [50]:
provider_percentages = df['provider'].value_counts(normalize=True) * 100
print("Percentage of rows for each unique provider:")
print(provider_percentages.to_string())

Percentage of rows for each unique provider:
provider
AWS      90.985678
Azure     9.014322
